In [2]:
from ola_cb import * 
from ola_RNN import * 
from ola_models import * 

import os, time, copy, math, re, json, pickle, random
import numpy as np
import pandas as pd

import torch, torchvision
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.ticker as ticker

from functools import partial 

cuda_available = torch.cuda.is_available()
device = torch.device("cuda:0" if cuda_available else "cpu")
print(f'''using device {device}''')

path = !pwd
path = path[0]
print(path)

using device cuda:0
/home/r2/Documents/RNNexp


In [ ]:
class SGRU(nn.Module):
    def __init__(self, in_sz, hd_sz, n_stacks):
        super(SGRU,self).__init__()
        self.in_sz = in_sz
        self.hd_sz = hd_sz
        self.n_stacks = n_stacks
        self.GRUs = []
        for i in range(n_stacks):
            self.GRUs.append(GRU(in_sz,hd_sz))
                
    def batch_forward(self,xb,yb,hd):
        # xb,yb,hd should be tensors     
        # I want preds and hidden outputs to have size [sql, n_stacks, _]
        preds = torch.zeros(size(xb,1), self.n_stacks, self.in_sz)
        hidds = torch.zeros(size(xb,1)+1, self.n_stacks, self.hd_sz)
        hidds[0] = hd
        for char in range(size(preds,1)):
            gru = self.GRUs[char]            
            preds[char],hidds[char+1] = gru.forward(xb[char],yb[char],hd[char])
            for stack in range(1,self.n_stacks):
                

class GRU(nn.Module):
    def __init__(self, in_sz, hd_sz):
        super(GRU,self).__init__()
        self.in_sz = in_sz
        self.hd_sz = hd_sz
        
        self.h_lin = nn.Linear(self.hd_sz,3*self.hd_sz)
        self.x_lin = nn.Linear(self.in_sz,3*self.hd_sz)        
        
        self.up_sig = nn.Sigmoid()
        self.re_sig = nn.Sigmoid()
            
        self.o1      = nn.Linear(self.hd_sz+self.in_sz,self.in_sz)  

        self.softmax = nn.LogSoftmax(dim=1)   
        self.loss    = 0 
            
    def forward(self,input,hidden):        
        x = self.x_lin(input)        
        h = self.h_lin(hidden)        
        x_u,x_r,x_n = x.chunk(3,1)
        h_u,h_r,h_n = h.chunk(3,1)
        update_gate = self.up_sig(x_u+h_u)        
        reset_gate  = self.re_sig(x_r+h_r)
        new_gate    = torch.tanh(x_n + reset_gate * h_n)         
        h_new       = update_gate * hidden + (1 - update_gate) * new_gate 
        
        combined   = torch.cat((input,h_new),1)
        combined   = self.o1(combined)

        prediction = self.softmax(combined)
        
        return prediction, h_new
    
    def batch_forward(self,xb,yb,hidden,loss_fn):
        self.train()
        if xb[0,0,1].item() == 1: hidden = self.initHidden(xb.shape[0])                   
        loss = 0 
        for char in range(xb.shape[1]):
            x,y           = xb[:,char],yb[:,char]
            x,y,hidden    = unpad(x,y,hidden)
            if x.shape[0] == 0: break
            output,hidden = self.forward(x,hidden)
            loss += loss_fn(output,y)    
        return output,hidden.detach(),loss/(char+1)

    def initHidden(self, bs):
        return cuda(torch.zeros(bs,self.hd_sz))